In [19]:

#------TRAINING & TESTING DATASET CREATION--------------
import xlrd, re


path = ("Tweets.xls") 
spreadSheet = xlrd.open_workbook(path)

tweetsData = []
sentiment = [] 
targetSentiment = ["positive", "negative","neutral"]

sheet = spreadSheet.sheet_by_index(1) 
for i in range(1, 420):
    arrStr = re.sub('[!@#\"$]', "", sheet.cell_value(i, 10)) 
    arrStr = re.sub(r"\//\S+", "", arrStr)
    tweetsData.append(arrStr) 
    sentiment.append(sheet.cell_value(i, 1))
    

def calScore(X_train, X_test, Y_train, Y_test):
    #--------------MODEL CREATION---------------
    from sklearn.feature_extraction.text import CountVectorizer
    from sklearn.feature_extraction.text import TfidfTransformer
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.pipeline import make_pipeline

    countVector = CountVectorizer()
    vocab= countVector.fit(X_train)
    # print("\nVocabulary: " + str(countVector.vocabulary_) + '\n')
    # print("Features: " + str(countVector.get_feature_names()) + '\n') 

    termCountMatrix = countVector.transform(X_train)
    print("SHAPE: "+ str(termCountMatrix.shape)) #3 documents and 7 unique terms/vocab
    # print("COUNT MATRIX: \n"+ str(termCountMatrix.toarray())) 

    #terms given tfIdf
    tfIdfTransformer = TfidfTransformer()
    tfIdfTransformer.fit(termCountMatrix)#sorted in the order of Features array
    # print("TFIDF:" + str(tfIdfTransformer.idf_))

    tdIdfMatrix = tfIdfTransformer.transform(termCountMatrix)
    # print("TFIDF:" + str(tdIdfMatrix.toarray()))

    model = MultinomialNB().fit(tdIdfMatrix,Y_train)
    # model = make_pipeline(TfidfVectorizer(), MultinomialNB())
    # model.fit(trainDict.values(), trainDict.keys())
    # labels = model.predict(testDict.values())

    #-------------------TESTING-----------------------
    newVectorMatrix = countVector.transform(X_test)
    newTFMatrix =  tfIdfTransformer.transform(newVectorMatrix)
    predicted = model.predict(newTFMatrix)
#     print("PREDICT",predicted)

    #TEST RESULTS: 
    #without stop word removal
    # test_size =0.3 ---> accuracy: 0.677595, 0.6673497, 0.66074, 0.67668
    # test_size =0.2 ---> 0.659153, 0.6639344, 0.67657,0.67725409, 0.6851092, 0.6803278, 0.691598
    # test_size =0.4 ---> 0.6622267759562842, 0.67400, 0.6816939890710383, 0.6798
    # test_size =0.5 ---> 0.6643442622950819, 0.671, 0.6653005, 0.6569
    # test_size =0.8 ---> 0.6534323770491803

    from sklearn import metrics

    accuracy = metrics.accuracy_score(Y_test,predicted)
    print("accuracy:", accuracy)
    print(metrics.classification_report(Y_test, predicted, targetSentiment))
    metrics.confusion_matrix(Y_test,predicted)
    
 
    # DATA SPLIT
    # from sklearn.model_selection import train_test_split
    # X_train, X_test, Y_train, Y_test = train_test_split(tweetsData, sentiment, test_size =0.2)
    # calScore(X_train, X_test, Y_train, Y_test)



# ---------KFold cross validation
from sklearn.model_selection import KFold
import numpy as np
kf  = KFold(n_splits=3)
kf
# for train_index, test_index in kf.split([1,2,3,4,5,6,7,8,9]):
#     print(train_index, test_index)
     
for train_index, test_index in kf.split(tweetsData):   
    X_train = np.array(tweetsData)[train_index]
    Y_train = np.array(tweetsData)[train_index]
    X_test = np.array(sentiment)[test_index]
    Y_test = np.array(sentiment)[test_index]
#     i=1
#     print("KFOLD:", i, ": ", X_test)
    calScore(X_train, X_test, Y_train, Y_test)    
#     i=i+1
     

SHAPE: (279, 1243)
accuracy: 0.0
              precision    recall  f1-score   support

    positive       0.00      0.00      0.00        46
    negative       0.00      0.00      0.00         0
     neutral       0.00      0.00      0.00        94

   micro avg       0.00      0.00      0.00       140
   macro avg       0.00      0.00      0.00       140
weighted avg       0.00      0.00      0.00       140

SHAPE: (279, 1192)
accuracy: 0.0
              precision    recall  f1-score   support

    positive       0.00      0.00      0.00        27
    negative       0.00      0.00      0.00        86
     neutral       0.00      0.00      0.00        27

   micro avg       0.00      0.00      0.00       140
   macro avg       0.00      0.00      0.00       140
weighted avg       0.00      0.00      0.00       140

SHAPE: (280, 1242)
accuracy: 0.0
              precision    recall  f1-score   support

    positive       0.00      0.00      0.00       139
    negative       0.00      0

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/s